In [ ]:
!pip3 install icalendar

In [ ]:
import pandas as pd

url = "http://www.cpliege.be/clubs/club0854.asp"

df = pd.read_html(url, header=5)[0]


df 

In [ ]:
agenda = df.copy()

# remove if column "Unnamed: 7" is empty OR starts with "(" and ends with ")"
agenda = agenda[~(agenda["Unnamed: 7"].isnull() | agenda["Unnamed: 7"].str.startswith("(") & agenda["Unnamed: 7"].str.endswith(")"))]

# for event in agenda.iterrows():
#     # if 7th column is not empty and does not starts with "(" and ends with ")"
#     if event[1][7] and event[1][7][0] != "(" and event[1][7][-1] != ")":
#         print(event[1][7])
    
print(len(agenda))

# rename columns
agenda.columns = ["Code", "Unknown", "Weekday", "Date", "Hour", "Équipe 1", "Équipe 2", "Catégorie", "Autre"]

# drop "Unknown" column
agenda.drop("Unknown", axis=1, inplace=True)

# replace . in hour by :
agenda["Hour"] = agenda["Hour"].str.replace(".", ":")

# remove Weekday column
agenda.drop("Weekday", axis=1, inplace=True)

# Date as datetime
agenda["Date"] = pd.to_datetime(agenda["Date"], format="%d/%m/%y")

# order by catégorie and then by date
agenda.sort_values(by=["Catégorie", "Date"], inplace=True)

# select U10
# agenda = agenda[agenda["Catégorie"] == "U 10 - MM F"]

agenda

# remove if Data is NaT
agenda = agenda[~agenda["Date"].isnull()]

agenda.to_excel("./data/agenda.xlsx", index=False)


In [ ]:
from icalendar import Calendar, Event
import uuid
from datetime import datetime
import tempfile, os

for event in agenda.iterrows():
    name = event[1]["Catégorie"] + ': '+ event[1]["Équipe 1"] + " vs " + event[1]["Équipe 2"]
    
    startime = pd.to_datetime(event[1]["Date"]).strftime("%Y-%m-%d") + " " + event[1]["Hour"]
    
    endtime = pd.to_datetime(startime) + pd.Timedelta(minutes=90)
    
    # set brussels timezone
    startime = pd.to_datetime(startime).tz_localize('Europe/Brussels')
    endtime = pd.to_datetime(endtime).tz_localize('Europe/Brussels')
    
    e.begin = startime.strftime("%Y-%m-%d %H:%M:%S")
    e.end = endtime.strftime("%Y-%m-%d %H:%M:%S")
    
    # location
    # location = event[1]["Équipe 1"]
    location = ""
    
    filename = "./data/" + event[1]["Catégorie"] + " - " + event[1]["Équipe 1"] + " vs " + event[1]["Équipe 2"] + ".ics"
    
    cal = Calendar()
    cal.add('prodid', 'Made with ❤️ by Martin Erpicum')
    cal.add('version', '2.0')
    cal.add('method', "PUBLISH")
    cal.add('X-WR-TIMEZONE', "Europe/Brussels")
    
    e = Event()
    e.add('summary', name)
    e.add('dtstart', startime)
    e.add('dtend', endtime)
    e.add('dtstamp', datetime.now())
    e.add('location', location)
    e.add('priority', 5)
    e.add('sequence', 1)
    e.add('description', name)
    e.add('url', url)
    e.add('uid', str(uuid.uuid4()))
    cal.add_component(e)
    
    # save to file
    f = open(filename, 'wb')
    f.write(cal.to_ical())
    f.close()
        
